# Video Game Consoles Analysis

## Introduction

## Requirements

## Data Collection And Export

The bulk of the data for this was collected using a web scraper from https://www.videogameconsolelibrary.com.

The originally scraped data was saved to a CSV file called 01-original-data.csv, which is stored in the data folder.

Additionally, a number of features were added to the data using Excel from a number of sources. These sources include:

- https://www.computinghistory.org.uk/ - Used for additional stats, such as launch price and release / discontinuation dates.
- https://www.macrotrends.net/ - Used for currency conversions of older systems reported in non-USD currencies.
- https://necretro.org/ - Used for additional stats, such as launch price and release / discontinuation dates.
- https://en.wikipedia.org/ Used for additional stats, such as launch price and release / discontinuation dates.

Lastly, the failure rate information for the xBox 360, PlayStation 3 and Wii were obtained from the below link:
https://www.squaretrade.com/htm/pdf/SquareTrade_Xbox360_PS3_Wii_Reliability_0809.pdf

The failure rate for other consoles is unknown so an estimate of 7.5% was used. This was determined by taking the failure rate of the Nintendo Wii (2.7%), which was rounded to 3% and the failure rate of the PlayStation 3 (10%) and working out the middle point between them (7.5%).

The final data that will be used for this is stored in `./data/02-processed-data.csv`. An Excel version (`02-processed-data.xlsx`) is also stored in the same location.

## Data Dictionary

1. platform - The name of the console / system.
2. north_america_original - The originally scraped sales numbers for North America (shown in millions sold).
3. north_america - The multiplying of north_america_original by one million.
4. europe_original - The originally scraped sales numbers for Europe (shown in millions sold).
5. europe - The multiplying of europe_original by one million.
6. japan_original - The originally scraped sales numbers for Japan (shown in millions sold).
7. japan - The multiplying of japan_original by one million.
8. rest_of_the_world_original - The originally scraped sales numbers for the rest of the world (shown in millions sold).
9. rest_of_the_world - The multiplying of rest_of_the_world_original by one million.
10. unknown_sales - The sales that were made that were not accounted for in any of the geographies above. This was determined by deducting the known geographies sales from the global feature.
11. global_original - The originally scraped total sales numbers (shown in millions sold).
12. global - The multiplying of global_original by one million.
13. released_date - The year the console / system was released.
14. discontinued_date - The year the console / system was discontinued from sale.
15. msrp_usd - The launch price of the console / system.
16. handheld - Is the console / system a handheld device. True = Yes, False = No.
17. total_available_games - The total games that were / are available for the console / system.
18. failure_rate_percentage - The failure rate of the console / system.
19. vendor - The vendor of the console / system.
20. position_when_scraped - The position the console / system was in at the time of being scraped. This is based on units sold.

### 1. Import The Required Libraries / Modules

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb